<a href="https://colab.research.google.com/github/sradical/ClinicalBert-Entity-Extraction/blob/main/ClinicalBertTraining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
!ls

drive  sample_data
